In [1]:
import pandas as pd
from pathlib import Path

In [2]:
import numpy as np
from collections import Counter

In [52]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [53]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "Uploading_Data_Colab_1.xlsx" with length 9000 bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving clean_feature_set.csv to clean_feature_set (2).csv
Saving clean_wait_times.csv to clean_wait_times (2).csv
User uploaded file "Uploading_Data_Colab_1.xlsx" with length 9000 bytes
User uploaded file "Uploading_Data_Colab_1.xlsx" with length 9000 bytes


In [3]:
wait_times_path = Path('../clean_wait_times.csv')
wait_times_df = pd.read_csv(wait_times_path)
wait_times_df.head()

,date,ride_name,sactmin
0,2015-01-01,7_dwarfs_train,54.5
1,2015-01-01,big_thunder_mtn,37.0
2,2015-01-01,haunted_mansion,27.5
3,2015-01-01,it_s_a_small_world,15.0
4,2015-01-01,splash_mountain,11.5


In [4]:
feature_set_path = Path('../clean_feature_set.csv')
feature_set_df = pd.read_csv(feature_set_path)
feature_set_df.head()

,date,day_week,day_year,week_year,month_year,year,prox_to_holiday,wdw_mean_temp,wdw_precip,schools_in_session,florida_schools_in_session
0,2015-01-01,5,0,0,1,2015,0,66.41,0.12,0.00,0.00
1,2015-01-02,6,1,0,1,2015,2,69.36,0.12,0.00,0.00
2,2015-01-03,7,2,0,1,2015,3,75.22,0.07,0.00,0.00
3,2015-01-04,1,3,1,1,2015,4,75.95,0.12,0.67,0.79
4,2015-01-05,2,4,1,1,2015,5,64.60,0.12,0.67,0.79


In [5]:
merged_data = wait_times_df.merge(feature_set_df, how='right', on='date')

In [6]:
merged_data.head()

,date,ride_name,sactmin,day_week,day_year,week_year,month_year,year,prox_to_holiday,wdw_mean_temp,wdw_precip,schools_in_session,florida_schools_in_session
0,2015-01-01,7_dwarfs_train,54.5,5,0,0,1,2015,0,66.41,0.12,0.0,0.0
1,2015-01-01,big_thunder_mtn,37.0,5,0,0,1,2015,0,66.41,0.12,0.0,0.0
2,2015-01-01,haunted_mansion,27.5,5,0,0,1,2015,0,66.41,0.12,0.0,0.0
3,2015-01-01,it_s_a_small_world,15.0,5,0,0,1,2015,0,66.41,0.12,0.0,0.0
4,2015-01-01,splash_mountain,11.5,5,0,0,1,2015,0,66.41,0.12,0.0,0.0


In [7]:
merged_data.dtypes

date                           object
ride_name                      object
sactmin                       float64
day_week                        int64
day_year                        int64
week_year                       int64
month_year                      int64
year                            int64
prox_to_holiday                 int64
wdw_mean_temp                 float64
wdw_precip                    float64
schools_in_session            float64
florida_schools_in_session    float64
dtype: object

In [8]:
merged_data['date'] = pd.to_datetime(merged_data['date'])

In [9]:
merged_data.dtypes

date                          datetime64[ns]
ride_name                             object
sactmin                              float64
day_week                               int64
day_year                               int64
week_year                              int64
month_year                             int64
year                                   int64
prox_to_holiday                        int64
wdw_mean_temp                        float64
wdw_precip                           float64
schools_in_session                   float64
florida_schools_in_session           float64
dtype: object

In [59]:
merged_data.describe()

,sactmin,day_week,day_year,week_year,month_year,year,prox_to_holiday,wdw_mean_temp,wdw_precip,schools_in_session,florida_schools_in_session
count,13210.000000,13217.000000,13217.000000,13217.000000,13217.000000,13217.000000,13217.000000,13217.000000,13217.000000,13217.000000,13217.000000
mean,22.542997,4.022547,183.341379,26.291065,6.564198,2017.214118,7.557842,73.665223,0.141189,0.682258,0.694854
std,15.805073,2.010890,109.211865,15.607384,3.571326,1.578649,6.515731,9.017332,0.076805,0.337521,0.409953
min,0.000000,1.000000,0.000000,0.000000,1.000000,2015.000000,0.000000,39.750000,0.030000,0.000000,0.000000
25%,12.333333,2.000000,86.000000,12.000000,3.000000,2016.000000,3.000000,68.320000,0.080000,0.440000,0.280000
50%,19.000000,4.000000,182.000000,26.000000,7.000000,2017.000000,6.000000,75.490000,0.120000,0.870000,0.970000
75%,28.372159,6.000000,282.000000,40.000000,10.000000,2019.000000,10.000000,81.070000,0.200000,0.940000,1.000000
max,176.000000,7.000000,365.000000,53.000000,12.000000,2020.000000,33.000000,87.760000,0.350000,0.970000,1.000000


In [75]:
merged_data_dropna = merged_data.dropna(how='any')

In [76]:
merged_data.corr(method='pearson')

,sactmin,day_week,day_year,week_year,month_year,year,prox_to_holiday,wdw_mean_temp,wdw_precip,schools_in_session,florida_schools_in_session
sactmin,1.000000,0.007300,-0.068398,-0.069782,-0.069770,0.138670,-0.056810,-0.074948,-0.029318,-0.068463,-0.070814
day_week,0.007300,1.000000,-0.010074,-0.028458,-0.010580,-0.001163,0.013281,-0.000486,0.009855,0.036729,0.026041
day_year,-0.068398,-0.010074,1.000000,0.999654,0.996733,0.021640,0.119195,0.256026,0.063588,-0.113431,-0.075776
week_year,-0.069782,-0.028458,0.999654,1.000000,0.996384,0.012107,0.119089,0.256214,0.063568,-0.114030,-0.075953
month_year,-0.069770,-0.010580,0.996733,0.996384,1.000000,0.022323,0.119184,0.259773,0.066193,-0.106497,-0.070504
year,0.138670,-0.001163,0.021640,0.012107,0.022323,1.000000,-0.052749,-0.101659,-0.056330,-0.006518,-0.008062
prox_to_holiday,-0.056810,0.013281,0.119195,0.119089,0.119184,-0.052749,1.000000,0.293649,0.341390,-0.119896,-0.136071
wdw_mean_temp,-0.074948,-0.000486,0.256026,0.256214,0.259773,-0.101659,0.293649,1.000000,0.597997,-0.293198,-0.264422
wdw_precip,-0.029318,0.009855,0.063588,0.063568,0.066193,-0.056330,0.341390,0.597997,1.000000,-0.497436,-0.491789
schools_in_session,-0.068463,0.036729,-0.113431,-0.114030,-0.106497,-0.006518,-0.119896,-0.293198,-0.497436,1.000000,0.948667


In [15]:
export_path = Path("../merged_data.json")

In [14]:
merged_data.to_json(export_path)

In [79]:
target = ["sactmin"]

Split Data into Training and Testing

In [85]:
#create features
X = (merged_data_dropna.drop(columns=['sactmin','date','ride_name']))


#create target
y = merged_data_dropna.loc[:,target].copy()

In [86]:
X.describe

<bound method NDFrame.describe of        day_week  day_year  week_year  month_year  year  prox_to_holiday  \
0             5         0          0           1  2015                0   
1             5         0          0           1  2015                0   
2             5         0          0           1  2015                0   
3             5         0          0           1  2015                0   
4             5         0          0           1  2015                0   
...         ...       ...        ...         ...   ...              ...   
13212         5       365         52          12  2020                0   
13213         5       365         52          12  2020                0   
13214         5       365         52          12  2020                0   
13215         5       365         52          12  2020                0   
13216         5       365         52          12  2020                0   

       wdw_mean_temp  wdw_precip  schools_in_session  \
0        

In [87]:
X.tail()

,day_week,day_year,week_year,month_year,year,prox_to_holiday,wdw_mean_temp,wdw_precip,schools_in_session,florida_schools_in_session
13212,5,365,52,12,2020,0,60.38,0.08,0.0,0.0
13213,5,365,52,12,2020,0,60.38,0.08,0.0,0.0
13214,5,365,52,12,2020,0,60.38,0.08,0.0,0.0
13215,5,365,52,12,2020,0,60.38,0.08,0.0,0.0
13216,5,365,52,12,2020,0,60.38,0.08,0.0,0.0


In [88]:
#check the balance of target values
y['sactmin'].value_counts()

15.000000    225
13.000000    218
16.000000    217
18.000000    211
21.000000    207
            ... 
56.400000      1
74.333333      1
63.600000      1
11.454545      1
23.700000      1
Name: sactmin, Length: 1266, dtype: int64

In [93]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_train.shape

(9907, 10)